In [1]:
import sys
import django
import os
file_dir = "/Users/mirbilal/Desktop/minsir/"
if file_dir not in sys.path:
    sys.path.insert(0, file_dir)

os.environ["DJANGO_SETTINGS_MODULE"] = "minsirx.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" 
django.setup()

import imaplib
import email
import pandas as pd
from datetime import datetime
import csv
from pandas import DataFrame

from email.header import decode_header
from bs4 import BeautifulSoup
import re
from apps.email_manager.models import EmailData, EmailAttachment
from django.core.files.base import ContentFile
import pytz


In [2]:

from os import environ


def clean_text(text):
    text = text.strip()
    text = re.sub(r'\n+', '\n', text)
    return text

def extract_main_body(email_body):
    main_body = re.split(r'From:\s', email_body, maxsplit=1)[0]
    return main_body

def html_to_text(html_content):
    soup = BeautifulSoup(html_content, "html.parser")
    for br in soup.find_all("br"):
        br.replace_with("\n")
    for p in soup.find_all("p"):
        p.insert_after("\n\n")
    for div in soup.find_all("div"):
        div.insert_after("\n")
    text = soup.get_text(separator="\n")
    return text

def decode_payload(payload, charsets):
    for charset in charsets:
        try:
            return payload.decode(charset)
        except (UnicodeDecodeError, LookupError):
            continue
    return payload.decode('latin1', errors='ignore')

def download_attachments_with_filename(email_id, email_password):
    mail = imaplib.IMAP4_SSL("outlook.office365.com", 993)
    mail.login(email_id, email_password)
    mail.select("Inbox_1")

    since_date = datetime(2023, 4, 1)
    since_date_formatted = since_date.strftime('%d-%b-%Y')
    result, data = mail.search(None, '(SINCE {date})'.format(date=since_date_formatted))
    email_nums = data[0].split()

    for num in email_nums:
        try:
            result, data = mail.fetch(num, '(RFC822)')
            raw_email = data[0][1]
            if isinstance(raw_email, int):
                continue
            msg = email.message_from_bytes(raw_email)

            # Decode the subject
            encoded_subject = msg.get("Subject")
            if encoded_subject:
                decoded_subject, encoding = decode_header(encoded_subject)[0]
                if encoding:
                    decoded_subject = decoded_subject.decode(encoding)
            else:
                decoded_subject = ""

            # Decode the 'From' field
            from_field = msg.get("From")
            if from_field:
                decoded_from, encoding = decode_header(from_field)[0]
                if encoding:
                    decoded_from = decoded_from.decode(encoding)
            else:
                decoded_from = ""

            # Decode the 'To' field
            to_field = msg.get("To")
            if to_field:
                decoded_to, encoding = decode_header(to_field)[0]
                if encoding:
                    decoded_to = decoded_to.decode(encoding)
            else:
                decoded_to = ""

            # Extract the email body
            email_body = ""
            for part in msg.walk():
                if part.get_content_type() == "text/plain":
                    charset = part.get_content_charset() or 'utf-8'
                    email_body = decode_payload(part.get_payload(decode=True), [charset])
                elif part.get_content_type() == "text/html":
                    charset = part.get_content_charset() or 'utf-8'
                    html_content = decode_payload(part.get_payload(decode=True), [charset])
                    email_body = html_to_text(html_content)

            email_body = clean_text(email_body)
            email_body = extract_main_body(email_body)

            # Process the email date
            date_str = msg.get("Date")
            date_obj = None
            if date_str:
                splitted_1 = date_str.split("+")
                sub_date = splitted_1[0].strip()
                splitted_2 = sub_date.split("-")
                fnal_date = splitted_2[0].strip()
                fnal_date = fnal_date.replace("GMT", "").strip()
                frmts = ["%a, %d %b %Y %H:%M:%S", "%d %b %Y %H:%M:%S"]
                is_dt_made = False
                for a_dt_frmt in frmts:
                    try:
                        date_obj = datetime.strptime(fnal_date, a_dt_frmt)
                        is_dt_made = True
                        break
                    except ValueError:
                        pass
                if not is_dt_made:
                    raise ValueError(f"unable to format date: {fnal_date}")
            else:
                date_obj = None
        
            if date_obj is not None:
                date_obj = pytz.utc.localize(date_obj)

            # Save email data
            email_data = EmailData(
                from_email=decoded_from,
                to_emails=[decoded_to],
                date=date_obj,
                subject=decoded_subject,
                body=email_body
            )
            email_data.save()

            # Process attachments
            if msg.get_content_maintype() == 'multipart':
                for part in msg.walk():
                    if part.get_content_maintype() == 'multipart':
                        continue
                    if part.get('Content-Disposition') is None:
                        continue
                    filename = part.get_filename()
                    if filename:
                        attachment_content = part.get_payload(decode=True)
                        email_attachment = EmailAttachment(
                            email=email_data,
                            name=filename
                        )
                        email_attachment.file.save(filename, ContentFile(attachment_content))
                        email_attachment.save()
        except Exception as e:
            # Print the error, email subject, and email date
            print(f"Error processing email number {num}: {e}")
            print(f"Subject: {encoded_subject if 'encoded_subject' in locals() else 'Unknown'}")
            print(f"Date: {date_str if 'date_str' in locals() else 'Unknown'}")

    mail.logout()

# Example usage
email_id = os.environ["MINSIR_EMAIL_ID"]
email_password = os.environ["MINSIR_EMAIL_PASSWORD"]
download_attachments_with_filename(email_id, email_password)



Error processing email number b'2554': unknown encoding: unknown-8bit
Subject: RE: Compliance Documents for Inventory Insurance (Eleganz Luxury)
Date: Wed, 25 Oct 2023 10:17:18 +0000
Error processing email number b'2897': object of type 'NoneType' has no len()
Subject: Fwd: AICL Ref # CL-0623-202504-S35-000042 | PAKISTAN PETROLEUM
 LIMITED | IR-02-ABDUL LATEEF BUGTI - WCT-SECURITY DEPARTMENT - E. # 1248 |
 Policy # PL-0822-202504-S35-000045 | ARA Ref # 33536
Date: Thu, 2 Nov 2023 07:24:17 +0000
Error processing email number b'3398': 'utf-8' codec can't encode characters in position 225-226: surrogates not allowed
Subject: Undeliverable: Fwd: Business figures of sika Pakistan
Date: Tue, 21 Nov 2023 06:08:00 +0000
Error processing email number b'16700': command: FETCH => Connection closed. 14
Subject: RE: GERRY'S INTERNATIONAL GROUP OF COMPANIES - HEALTH PROPOSAL
 REQUIRED. 
Date: Thu, 17 Aug 2023 06:06:16 +0000
Error processing email number b'16701': Connection closed. 14
Subject: RE: G